In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from astropy.units import one
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import itertools
import random
import math
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score, \
                            precision_recall_curve, average_precision_score, roc_auc_score, roc_curve, auc, accuracy_score

from IPython.display import clear_output

from tqdm import tqdm_notebook

import pandas as pd
# the distribution of sequence length
from matplotlib import pyplot as plt

In [2]:
# from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
# from torch.nn.utils.rnn import pad_sequence
# from torch import Tensor

# import torch
# import torch.nn.functional as F
# import torch.nn as nn
# import torch.utils.data as data_utils
# import torch.optim.lr_scheduler as lr_scheduler 
# import matplotlib.pyplot as plt
# import copy

In [3]:
import lightgbm as lgb
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score


import pandas as pd
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import matplotlib.pyplot as plt

# sklearn tools for model training and assesment
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.metrics import (roc_curve, auc, accuracy_score, balanced_accuracy_score, matthews_corrcoef)

In [4]:
np.random.seed(0)


## paths

In [5]:
project_dir = '../'

#ds_dir = f'{project_dir}data/seq/'
ds_dir = f'{project_dir}data/processed/'

snapshot_dir = f'{project_dir}snapshots/'

classes = ['lnc_e','lnc_p']
class_suffix = '_noN'

neg_filename = f'{ds_dir}train_val_test__80_20/{classes[0]}{class_suffix}_tr_val.csv'
pos_filename = f'{ds_dir}train_val_test__80_20/{classes[1]}{class_suffix}_tr_val.csv'

test_neg_filename = f'{ds_dir}train_val_test__80_20/{classes[0]}{class_suffix}_test.csv'
test_pos_filename = f'{ds_dir}train_val_test__80_20/{classes[1]}{class_suffix}_test.csv'


# feature_filenames_class_0 = [f'{ds_dir}train_val_test/{classes[0]}{class_suffix}_1mer_features_tr_val.csv',
#                              f'{ds_dir}train_val_test/{classes[0]}{class_suffix}_2mer_features_tr_val.csv',
#                              f'{ds_dir}train_val_test/{classes[0]}{class_suffix}_3mer_features_tr_val.csv']
# feature_filenames_class_1 = [f'{ds_dir}train_val_test/{classes[1]}{class_suffix}_1mer_features_tr_val.csv',
#                              f'{ds_dir}train_val_test/{classes[1]}{class_suffix}_2mer_features_tr_val.csv',
#                              f'{ds_dir}train_val_test/{classes[1]}{class_suffix}_3mer_features_tr_val.csv']


# test_feature_filenames_class_0 = [f'{ds_dir}train_val_test/{classes[0]}{class_suffix}_1mer_features_test.csv',
#                                   f'{ds_dir}train_val_test/{classes[0]}{class_suffix}_2mer_features_test.csv',
#                                   f'{ds_dir}train_val_test/{classes[0]}{class_suffix}_3mer_features_test.csv']
# test_feature_filenames_class_1 = [f'{ds_dir}train_val_test/{classes[1]}{class_suffix}_1mer_features_test.csv',
#                                   f'{ds_dir}train_val_test/{classes[1]}{class_suffix}_2mer_features_test.csv',
#                                   f'{ds_dir}train_val_test/{classes[1]}{class_suffix}_3mer_features_test.csv']


bio_feature_filenames_class_0 = [f'{ds_dir}train_val_test__80_20/{classes[0]}_nosim.fa.matrix_tr_val.csv']
bio_feature_filenames_class_1 = [f'{ds_dir}train_val_test__80_20/{classes[1]}_nosim.fa.matrix_tr_val.csv']


test_bio_feature_filenames_class_0 = [f'{ds_dir}train_val_test__80_20/{classes[0]}_nosim.fa.matrix_test.csv']
test_bio_feature_filenames_class_1 = [f'{ds_dir}train_val_test__80_20/{classes[1]}_nosim.fa.matrix_test.csv']



In [6]:
# load positive and negative sequence data to construct ground label list

with open(neg_filename,'r') as nf:
    neg_examples = [seq.strip() for seq in nf.readlines()]
    num_neg = len(neg_examples) # number of negative examples
    print(num_neg)

with open(pos_filename,'r') as pf:
    pos_examples = [seq.strip() for seq in pf.readlines()]
    num_pos = len(pos_examples) # number of positive examples
    print(num_pos)    

sequences_DS = neg_examples + pos_examples 
labels_DS = np.array([0]*num_neg + [1]*num_pos)   
len(sequences_DS), len(labels_DS)

scale_pos_weight = num_neg/num_pos

2312
5462


In [7]:
# load training + validation data

bio_features_class0 = np.concatenate([pd.read_csv(bio_feature_filename, header=None, index_col=False).to_numpy() for bio_feature_filename in bio_feature_filenames_class_0], axis = 1)
bio_features_class1 = np.concatenate([pd.read_csv(bio_feature_filename, header=None, index_col=False).to_numpy() for bio_feature_filename in bio_feature_filenames_class_1], axis = 1)

neg_bio_features = bio_features_class0
pos_bio_features = bio_features_class1

bio_features_DS = np.concatenate([bio_features_class0, bio_features_class1], axis = 0)    
print(neg_bio_features.shape, pos_bio_features.shape,bio_features_DS.shape)

(2312, 292) (5462, 292) (7774, 292)


In [8]:
# load test data

test_bio_features_class0 = np.concatenate([pd.read_csv(bio_feature_filename, header=None, index_col=False).to_numpy() for bio_feature_filename in test_bio_feature_filenames_class_0], axis = 1)
test_bio_features_class1 = np.concatenate([pd.read_csv(bio_feature_filename, header=None, index_col=False).to_numpy() for bio_feature_filename in test_bio_feature_filenames_class_1], axis = 1)

test_neg_bio_features = test_bio_features_class0
test_pos_bio_features = test_bio_features_class1

test_bio_features_DS = np.concatenate([test_bio_features_class0, test_bio_features_class1], axis = 0)    

test_labels_DS = np.array([0]*test_bio_features_class0.shape[0] + [1]*test_bio_features_class1.shape[0])  
print(test_neg_bio_features.shape, test_pos_bio_features.shape,test_bio_features_DS.shape, len(test_labels_DS))

(579, 292) (1366, 292) (1945, 292) 1945


In [9]:
def lgb_mcc_score(y_hat, data):
    y_true = data.get_label()
    y_pred = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'mcc', matthews_corrcoef(y_true, y_pred), True

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True


def compute_sens_spec_mcc(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    #cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
    tn, fp, fn, tp = cm.ravel()

    specificity = tn/(fp+tn)
    sensitivity = tp/(tp+fn)
    
    return {'specificity':specificity, 'sensitivity': sensitivity, 'mcc':matthews_corrcoef(y_true, y_pred)}

In [10]:
# randomly shuffle the dataset (required as all negative class examples come first in the original)

indx = list(range(len(labels_DS)))
np.random.shuffle(indx)
labels_DS_np = np.array(labels_DS)
labels_shuf = labels_DS_np[indx]
bio_features_shuf = bio_features_DS[indx,:]


train_frac = 0.8
val_frac = 1 - train_frac

train_count = int(len(indx)*train_frac)
val_count = len(indx) - train_count

train_features = bio_features_shuf[:train_count,:]
train_labels = labels_shuf[:train_count]

val_features = bio_features_shuf[train_count:,:]
val_labels = labels_shuf[train_count:]

test_features = test_bio_features_DS
test_labels = test_labels_DS

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import time



In [43]:
# k = 0
# skf = StratifiedKFold(n_splits=10)
# kernel = 'poly'
# C = 1
# results_CV = []
# for train_idx, val_idx in skf.split(bio_features_shuf, labels_shuf):
#     k += 1
#     train_features = bio_features_shuf[train_idx,:]
#     train_labels = labels_shuf[train_idx]

#     val_features = bio_features_shuf[val_idx,:]
#     val_labels = labels_shuf[val_idx]

#     start_time = time.time()
#     svm_clf = SVC(gamma='scale', kernel = kernel, C = C)
#     svm_clf.fit(train_features, train_labels)
#     end_time = time.time()
#     print(end_time-start_time, 'seconds')
    
#     val_preds = svm_clf.predict(val_features)
#     train_preds = svm_clf.predict(train_features)
#     #np.sum(val_preds[val_labels==1] == val_labels[val_labels==1])/len(val_labels[val_labels==1])

#     result = compute_sens_spec_mcc(val_labels, val_preds)
#     results_CV.append(list(result.values()) )
#     print(f'fold{k}:',result['mcc'])      

In [44]:
# result.values(), results_CV  
# np.mean(results_CV, axis=0)

# SVM Cross validation 

In [49]:
skf = StratifiedKFold(n_splits=5)

parameters = {'kernel':('rbf',), 'C':range(1, 10+1)}

print('                    specificity', 'sensitivity', 'mcc')
for kernel in parameters['kernel']:
    for C in parameters['C']:
        
        results_CV = []
        k = 0
        for train_idx, val_idx in skf.split(bio_features_shuf, labels_shuf):
            k += 1
            train_features = bio_features_shuf[train_idx,:]
            train_labels = labels_shuf[train_idx]

            val_features = bio_features_shuf[val_idx,:]
            val_labels = labels_shuf[val_idx]

            start_time = time.time()
            svm_clf = SVC(gamma='scale', kernel = kernel, C = C)
            svm_clf.fit(train_features, train_labels)
            end_time = time.time()
            #print(end_time-start_time, 'seconds')

            val_preds = svm_clf.predict(val_features)
            train_preds = svm_clf.predict(train_features)
            #np.sum(val_preds[val_labels==1] == val_labels[val_labels==1])/len(val_labels[val_labels==1])

            result = compute_sens_spec_mcc(val_labels, val_preds)
            results_CV.append(list(result.values()) )
            #print(f'fold {02d:k}:', result)  
            
        results_CV = np.array(results_CV)
        
        print(f"kernel={kernel}, C={C}:", np.mean(results_CV, axis=0) )
        
      

                    specificity sensitivity mcc
kernel=poly, C=1: [0.00519294 0.99816883 0.02677993]
kernel=poly, C=2: [0.00648977 0.9967038  0.02390304]
kernel=poly, C=3: [0.0082195  0.99523893 0.02213555]


KeyboardInterrupt: 

                    specificity sensitivity mcc
kernel=rbf, C=1: [0.82698849 0.84621601 0.64388982]
kernel=rbf, C=2: [0.82829093 0.8539043  0.65456531]
kernel=rbf, C=3: [0.82785897 0.85866386 0.66024144]
kernel=rbf, C=4: [0.8226679  0.86232552 0.66067237]
kernel=rbf, C=5: [0.82180397 0.86415602 0.66228625]
kernel=rbf, C=6: [0.82007798 0.86635332 0.66372415]
kernel=rbf, C=7: [0.81445495 0.86745105 0.66052932]
kernel=rbf, C=8: [0.81142745 0.86873326 0.6597708 ]
kernel=rbf, C=9: [0.81142932 0.86891625 0.6599914 ]
kernel=rbf, C=10: [0.81099642 0.86928238 0.66013526]


# SVM Testing

In [50]:
kernel_test = 'rbf'
kernel_C = 5

# SVM retrain
svm_clf = SVC(gamma='scale', kernel = kernel_test, C = kernel_C)
svm_clf.fit(bio_features_shuf, labels_shuf)

test_preds = svm_clf.predict(test_features)


test_result = compute_sens_spec_mcc(test_labels, test_preds)
print("test spec., sens. mcc:")
print(test_result['specificity'], test_result['sensitivity'], test_result['mcc'])



test spec., sens. mcc:
0.8117443868739206 0.8638360175695461 0.6533761798120308


### Reported results:
###                      specificity        sensitivity        mcc
### CV kernel=rbf, C=5: [0.82180397         0.86415602         0.66228625]
### Testing:             0.8117443868739206 0.8638360175695461 0.6533761798120308

# Logistic Regression

In [54]:

results_CV = []
k = 0
for train_idx, val_idx in skf.split(bio_features_shuf, labels_shuf):
    k += 1
    train_features = bio_features_shuf[train_idx,:]
    train_labels = labels_shuf[train_idx]

    val_features = bio_features_shuf[val_idx,:]
    val_labels = labels_shuf[val_idx]

    start_time = time.time()
    clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(train_features, train_labels)

    end_time = time.time()
    #print(end_time-start_time, 'seconds')

    val_preds = clf.predict(val_features)

    result = compute_sens_spec_mcc(val_labels, val_preds)
    results_CV.append(list(result.values()) )
    #print(f'fold {02d:k}:', result)  

results_CV = np.array(results_CV)

print(f":", np.mean(results_CV, axis=0) )

/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


: [0.70545567 0.88594167 0.59602041]


/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [55]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(bio_features_shuf, labels_shuf)

test_preds = clf.predict(test_features)
#np.sum(val_preds[val_labels==1] == val_labels[val_labels==1])/len(val_labels[val_labels==1])


test_result = compute_sens_spec_mcc(test_labels, test_preds)
print("validation spec., sens. mcc:")
print(test_result['specificity'], test_result['sensitivity'], test_result['mcc'])


validation spec., sens. mcc:
0.7150259067357513 0.8879941434846267 0.6067087324217663


/home/tarik/anaconda3/envs/project_selfie/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
